# Data-driven approach to identifying changes occurring in primary care activity during the pandemic

## Generating data for decile charts to show activity across all practices in TPP

See `decile_charts_[concept]` for charts

In [1]:
import pyodbc
import pandas as pd
import os
from IPython.display import display, Markdown

dbconn = os.environ.get('DBCONN', None)
if dbconn is None:
    display("No SQL credentials. Check that the file 'environ.txt' is present. Refer to readme for further information")
else:
    dbconn = dbconn.strip('"')

# import custom functions from 'lib` folder
import sys
sys.path.append('../lib/')
    
from functions import closing_connection, load_concept, process_df, join_concept_descriptions, get_subcodes

In [2]:
# global variables 
if 'OPENCoronaExport' in dbconn:
    threshold = 1 # min activity threshold
else:
    threshold = 1000

end_date = "20200930"

# Lookup/Reference tables

## Create table of CTV3 codes

In [3]:
sql = f'''select 
CASE WHEN CHARINDEX('.',CTV3Code) > 0
THEN LEFT(CTV3Code,CHARINDEX('.',CTV3Code)-1) 
ELSE CTV3Code END AS first_digits, 
Description
from CTV3Dictionary 
WHERE LEFT(CTV3Code,1) NOT IN ('.', '0') -- Read thesaurus, occupations
'''

pd.set_option('display.max_rows', 200)
with closing_connection(dbconn) as connection:
    codes = pd.read_sql(sql, connection).sort_values(by=["first_digits"])

codes.to_csv(os.path.join('..','data','code_dictionary.csv'))    

# add fields for later use
codes['concept_digit'] = codes.first_digits
codes['concept_desc'] = codes.Description

codes.head(2)


,first_digits,Description,concept_digit,concept_desc
0,1,History/symptoms,1,History/symptoms
1,11,Depth of history,11,Depth of history


## Load Concept map

In [4]:
display(Markdown("## First level concepts"))
concepts = load_concept("ctv3-buckets-1.json", codes)
display(concepts.loc[concepts["descendant_clean"]=="2469"])

display(Markdown("## Second level concepts"))
concepts2 = load_concept("ctv3-buckets-2.json", codes)
display(concepts2[["concept_digit", "concept_desc"]].drop_duplicates().head(2))

display(Markdown("## Third level concepts"))
concepts3 = load_concept("ctv3-buckets-3.json", codes)
display(concepts3[["concept_digit", "concept_desc"]].drop_duplicates().head(2))

## First level concepts

,ancestor,descendant,concept_digit,descendant_clean,concept_desc
29283,X74VQ,2469.,X74VQ,2469,Context-dependent categories


## Second level concepts

,concept_digit,concept_desc
0,02...,NaN
173,04...,NaN


## Third level concepts

,concept_digit,concept_desc
0,0242.,NaN
4,02A..,NaN


In [5]:
# find concept level 3 ancestors for laboratory tests/findings
concepts3.loc[concepts3["descendant"].isin(["42I..", # Differential white blood cell count
                                            "X77Wg" # thyroid function test
                                            ])]

,ancestor,descendant,concept_digit,descendant_clean,concept_desc
252742,X7A0B,X77Wg,X7A0B,X77Wg,Laboratory test
398777,X76sW,42I..,X76sW,42I,Laboratory test observations


# Population included

In [7]:
sql = f'''-- total patient and practice count
SELECT
COUNT(DISTINCT Organisation_ID) AS practice_count,
COUNT(DISTINCT Patient_ID) AS patient_count
FROM RegistrationHistory
WHERE StartDate <= '{end_date}' AND
EndDate >= '{end_date}' -- registrations which were live at the end of the study period
'''

with closing_connection(dbconn) as connection:
    df = pd.read_sql(sql, connection).transpose()
    
display(f"Practice count {df[0][0]}, Patient count {df[0][1]}")

'Practice count 2546, Patient count 23878341'

# Data Analysis

## Rank all "level 2" codes by frequency of appearance in 2020

In [9]:
# select events for selcted subset of codelist
sql = f'''select TOP 500 LEFT(CTV3Code,2) AS first_digits, COUNT(Patient_ID) as events, COUNT(DISTINCT Patient_ID) as patients, MAX(CAST(ConsultationDate AS DATE)) AS latest_date
FROM CodedEvent 
WHERE 
ConsultationDate IS NOT NULL 
AND CAST(LEFT(CTV3Code,1) AS VARCHAR) NOT IN ('.', '0', 'X', 'Y', 'U')
AND CAST(LEFT(CTV3Code,2) AS VARCHAR) NOT IN ('49') -- remove semen analysis
AND ConsultationDate >= '20200101'
AND ConsultationDate <= '{end_date}'
GROUP BY LEFT(CTV3Code,2)
HAVING COUNT(Patient_ID) > {threshold}
ORDER BY first_digits
'''

with closing_connection(dbconn) as connection:
    df_l2 = pd.read_sql(sql, connection).sort_values(by="events", ascending=False)
    

### Add extra columns and descriptions

In [21]:
top_l2 = process_df(df_l2, codes)
top_l2 = join_concept_descriptions(top_l2, concepts, concepts2, concepts3)

out = top_l2.drop("events", 1)
out["patients"] = (10*(out["patients"]/10).round(0)).astype(int)
out.to_csv(os.path.join("..","output","level_two_codes.csv"), index=False)
out.head(2)

,first_digits,patients,latest_date,2020 events (mill),2020 Patient count (mill),first_digit,digits,Description,concept_desc
0,42,4421920,2020-09-30,65.66,4.42,4,2,Haematology,Laboratory procedures
1,24,5328110,2020-09-30,20.22,5.33,2,2,Examination of cardiovascular system (& [vascu...,"Operations, procedures and interventions"


## Rank level 3 codes by frequency of appearance in 2020
These codes may be more useful than full-length codes because they contain all the various options for a particular activity (e.g. "birth details" includes "normal birth", "other delivery routes" etc. This therefore captures general trends in activity types rather than the detailed specifics. However, the codes beginning with X, Y, U do not have descriptions when truncated to level 3 so will be done in full sepatately. 

In [11]:
sql = f'''select TOP 500
LEFT(CTV3Code,3) AS first_digits, COUNT(Patient_ID) as events, COUNT(DISTINCT Patient_ID) as patients
FROM CodedEvent 
WHERE 
ConsultationDate IS NOT NULL 
AND CAST(LEFT(CTV3Code,1) AS VARCHAR) NOT IN ('.', '0', 'U', 'X','Y')
AND CAST(LEFT(CTV3Code,2) AS VARCHAR) NOT IN ('49') -- remove semen analysis
AND ConsultationDate >= '20200101'
AND ConsultationDate <= '{end_date}'
GROUP BY LEFT(CTV3Code,3)
HAVING COUNT(Patient_ID) > {threshold}
ORDER BY events DESC
'''

with closing_connection(dbconn) as connection:
    df_l3 = pd.read_sql(sql, connection).sort_values(by="events", ascending=False)

In [12]:
top_l3 = process_df(df_l3, codes)
top_l3 = join_concept_descriptions(top_l3, concepts, concepts2, concepts3)

out = top_l3.drop(["events"], axis=1)

out.head(2)

,first_digits,patients,2020 events (mill),2020 Patient count (mill),first_digit,digits,Description,concept_desc
0,246,5024913,16.97,5.02,2,3,O/E - blood pressure reading,Context-dependent categories
1,426,4217135,8.18,4.22,4,3,Red blood cell count,Laboratory procedures


## Rank codes starting with U, X or Y by frequency of appearance in 2020¶
X and Y codes are not neatly subgrouped/hierarchied but contain lots of interesting activity

In [13]:
sql = f'''select TOP 500
e.CTV3Code AS first_digits, 
COUNT(Patient_ID) as events,
COUNT(DISTINCT Patient_ID) as patients
FROM CodedEvent e
WHERE ConsultationDate IS NOT NULL 
AND CAST(LEFT(e.CTV3Code,1) AS VARCHAR) IN ('U', 'X', 'Y')
AND ConsultationDate >= '20200101'
AND ConsultationDate <= '{end_date}'
GROUP BY e.CTV3Code
HAVING COUNT(Patient_ID) > {threshold}
ORDER BY events DESC
'''

with closing_connection(dbconn) as connection:
    df_xy = pd.read_sql(sql, connection).sort_values(by="events", ascending=False)

In [14]:
top_xy = process_df(df_xy, codes)
top_xy = join_concept_descriptions(top_xy, concepts, concepts2, concepts3)

out = top_xy.drop(["events"], axis=1)

out.head(2)

,first_digits,patients,2020 events (mill),2020 Patient count (mill),first_digit,digits,Description,concept_desc
0,Y237d,20218079,84.47,20.22,Y,5,Additional SCR dataset uploaded under COPI Reg...,Administration
1,XaMiI,7500366,26.40,7.50,X,5,Short message service text message sent to pat...,Administration


## Create combined list of Level 3 codes and X,Y,U codes

In [20]:
combined_list = pd.concat([top_xy, top_l3], ignore_index=True).sort_values(by="2020 events (mill)", ascending=False).head(500)
out = combined_list.drop(["events"], axis=1)
out["patients"] = (10*(out["patients"]/10).round(0)).astype(int)

out.to_csv(os.path.join("..","output","combined_codelist.csv"), index=False)
out.head()

,first_digits,patients,2020 events (mill),2020 Patient count (mill),first_digit,digits,Description,concept_desc
0,Y237d,20218080,84.47,20.22,Y,5,Additional SCR dataset uploaded under COPI Reg...,Administration
1,XaMiI,7500370,26.40,7.50,X,5,Short message service text message sent to pat...,Administration
517,246,5024910,16.97,5.02,2,3,O/E - blood pressure reading,Context-dependent categories
2,XaIge,4208210,9.48,4.21,X,5,Telephone consultation,Administration
518,426,4217140,8.18,4.22,4,3,Red blood cell count,Laboratory procedures
